In [22]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [29]:
lr = 0.00001
batch_size = 64
display_epochs = 1
logs_path = '/tmp/tensorflow_logs/mnist_fully_connected'
num_steps = int(data.train.num_examples / batch_size)
num_epochs = 5
wh = 28
config = tf.ConfigProto(device_count = {'GPU': 1})

h1 = 512
h2 = 512
h3 = 512
inp = wh*wh
out = num_class = 10

X = tf.placeholder(tf.float32, shape=[None, inp], name="InputData")
Y = tf.placeholder(tf.float32, shape=[None, out], name="LabelsData")

In [30]:
W = {
    "h1": tf.Variable(tf.random_normal([inp, h1]), name="Weights_Input_to_H1"),
    "h2": tf.Variable(tf.random_normal([h1, h2]), name="Weights_H1_to_H2"),
    "h3": tf.Variable(tf.random_normal([h2, h3]), name="Weights_H2_to_H3"),
    "out": tf.Variable(tf.random_normal([h3, out]), name="Weights_H3_to_Output"),
}

b = {
    "h1": tf.Variable(tf.random_normal([h1]), name="Bias_Input_to_H1"),
    "h2": tf.Variable(tf.random_normal([h2]), name="Bias_H1_to_H2"),
    "h3": tf.Variable(tf.random_normal([h3]), name="Bias_H2_to_H3"),
    "out": tf.Variable(tf.random_normal([out]), name="Bias_H3_to_Output"),
}

In [31]:
layer1 = tf.add(tf.matmul(X, W["h1"]), b["h1"])
layer2 = tf.add(tf.matmul(layer1, W["h2"]), b["h2"])
layer3 = tf.add(tf.matmul(layer2, W["h3"]), b["h3"])
nn = out_layer = tf.add(tf.matmul(layer3, W["out"]), b["out"])

In [32]:
with tf.name_scope('Loss'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=nn, labels=Y
    ))

with tf.name_scope('SGD'):
    train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss_op)

with tf.name_scope('Accuracy'):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(nn, 1), tf.argmax(Y, 1)), tf.float32))

predictions = tf.argmax(nn, 1)

init = tf.global_variables_initializer()


training_loss = tf.summary.scalar("Training_Loss", loss_op)

validation_accuracy = tf.summary.scalar("Validation_Accuracy", accuracy)
validation_loss = tf.summary.scalar("Validation_Loss", loss_op)

test_accuracy = tf.summary.scalar("Test_Accuracy", accuracy)

training_summary_op = tf.summary.merge([training_loss])
test_summary_op = tf.summary.merge([test_accuracy])
validation_summary_op = tf.summary.merge([validation_loss, validation_accuracy])
preds = []

In [33]:
with tf.Session(config=config) as S:
    train_writer = tf.summary.FileWriter(logs_path + '/train', S.graph)
    validation_writer = tf.summary.FileWriter(logs_path + '/validation')
    test_writer = tf.summary.FileWriter(logs_path + '/test')

    S.run(init)
    
    for epoch in range(1, num_epochs + 1):
        avg_cost = 0.
        for step in range(1, num_steps + 1):
            batchX, batchY = data.train.next_batch(batch_size)

            _, c, summary =  S.run([train_op, loss_op, training_summary_op], feed_dict= { X: batchX, Y: batchY })
            avg_cost += c / num_steps
            train_writer.add_summary(summary, epoch * num_steps + step)

        vall_acc, val_loss, summary = S.run([accuracy, loss_op, validation_summary_op], feed_dict= { X: data.validation.images, Y: data.validation.labels })
        validation_writer.add_summary(summary, epoch)

        if epoch % display_epochs == 0 or epoch == 1:
            print("Epoch: ", '%04d' % epoch, "Cost: ", "{:.5f}".format(avg_cost), "Validation Accuracy: ", vall_acc)

    print("Optimization Finished!")

    test_acc, preds, summary = S.run([accuracy, predictions, test_summary_op], feed_dict= { X: data.test.images, Y: data.test.labels })
    test_writer.add_summary(summary, 1)
    print("Test Accuracy:", test_acc)


Epoch:  0001 Cost:  11678.90396 Validation Accuracy:  0.8584
Epoch:  0002 Cost:  5017.68791 Validation Accuracy:  0.8772
Epoch:  0003 Cost:  3874.01328 Validation Accuracy:  0.8788
Epoch:  0004 Cost:  3268.70871 Validation Accuracy:  0.8892
Epoch:  0005 Cost:  2820.18248 Validation Accuracy:  0.8834
Optimization Finished!
Test Accuracy: 0.881


In [1]:
# Run tensorboard --logdir=/tmp/tensorflow_logs
from visualization import create_atlas, create_facets
import numpy as np
from IPython.core.display import display, HTML

with tf.Session() as S:
    actual = np.array(tf.argmax(data.test.labels, 1).eval())

facets = create_facets(data.test.images, preds, actual, "mnist", "fully_connected")

display(HTML(facets))

NameError: name 'tf' is not defined

In [ ]:
# References
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/neural_network_raw.ipynb